In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy
import mpmath
from scipy import stats
from sklearn.neighbors import KernelDensity
import functools
import pandas as pd
import math
import random
import seaborn as sns
import itertools
from itertools import groupby
#import separation_algorithm as sepa
#import functions_eye_tracker_project as funcs
import sklearn
import os
from sklearn.neighbors import KernelDensity
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import copy
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.inspection import permutation_importance

In [ ]:
def return_index(array,value):
    for i in range(len(array)):
        if array[i] == value:
            return(i)
    

In [ ]:
experim_max = 26
part_max = 60
data = []
separation_threshold = 0
participant_list= []
current_part = 0
for part in range(1,part_max):
    for exper in range(1,experim_max):
        try:
            df = pd.read_excel('/Users/pedroreg/Desktop/research copy/Maryam_Eye-Tracking Dataset/ASD participant/'+str(part)+ ' ASD/E'+str(exper)+'('+str(part)+ ').xlsx',engine='openpyxl')
            df=df[df['Point of Regard Right X [px]'] !=0]
            df=df[df['Point of Regard Right Y [px]'] !=0]
            df= df[df['Category Right'].isin( ['Fixation', 'Saccade' ])]
            data.append(df)
            print('E'+str(exper)+'('+str(part)+ ')')
            separation_threshold +=1
            participant_list.append(part)
        except FileNotFoundError:
            0
            
        try:
            df = pd.read_excel('/Users/pedroreg/Desktop/research copy/Maryam_Eye-Tracking Dataset/TD participant/'+str(part)+ ' TD/E'+str(exper)+'('+str(part)+ ').xlsx',engine='openpyxl')
            df=df[df['Point of Regard Right X [px]'] !=0]
            df=df[df['Point of Regard Right Y [px]'] !=0]
            df= df[df['Category Right'].isin( ['Fixation', 'Saccade' ])]
            data.append(df)
            print('E'+str(exper)+'('+str(part)+ ')')
            participant_list.append(part)
        except FileNotFoundError:
            0



In [ ]:
separation_threshold2 = 28

In [ ]:
######## LOAD DATA

In [ ]:
T = [[] for i in range(len(np.unique(participant_list))) ]
X = [[] for i in range(len(np.unique(participant_list))) ]
XX=[[] for i in range(len(np.unique(participant_list))) ]
Y = [[] for i in range(len(np.unique(participant_list))) ]
YY=[[] for i in range(len(np.unique(participant_list))) ]
T = [[] for i in range(len(np.unique(participant_list))) ]
TT = [[] for i in range(len(np.unique(participant_list))) ]
TT_pupil = [[] for i in range(len(np.unique(participant_list))) ]
X = [[] for i in range(len(np.unique(participant_list))) ]
Y = [[] for i in range(len(np.unique(participant_list))) ]
d_X = [[] for i in range(len(np.unique(participant_list))) ]
d_Y = [[] for i in range(len(np.unique(participant_list))) ]
d_T = [[] for i in range(len(np.unique(participant_list))) ]
d_T_pupil = [[] for i in range(len(np.unique(participant_list))) ]
d_S = [[] for i in range(len(np.unique(participant_list))) ]
vS = [[] for i in range(len(np.unique(participant_list))) ]
vX = [[] for i in range(len(np.unique(participant_list))) ]
dX = [[] for i in range(len(np.unique(participant_list))) ]
dY = [[] for i in range(len(np.unique(participant_list))) ]
dS = [[] for i in range(len(np.unique(participant_list))) ]
dT = [[] for i in range(len(np.unique(participant_list))) ]
saccade = [[] for i in range(len(np.unique(participant_list))) ]
dX_lag = [[] for i in range(len(np.unique(participant_list))) ]
dY_lag = [[] for i in range(len(np.unique(participant_list))) ]
dS2_lag = [[] for i in range(len(np.unique(participant_list))) ]
dS4_lag = [[] for i in range(len(np.unique(participant_list))) ]
pupil = [[] for i in range(len(np.unique(participant_list))) ]


center_X = 1280/2
center_Y = 1024/2

for i in range(len(data)):

    if len(data[i])>100:
        
        temp_x = pd.to_numeric(data[i]['Point of Regard Right X [px]'])
        temp_y = pd.to_numeric(data[i]['Point of Regard Right X [px]'])
        temp_tt = pd.to_numeric(data[i]['RecordingTime [ms]'])

        temp_dtt = np.diff(temp_tt)

        distance = (temp_x**2 + temp_y**2)**0.5
        points_to_keep = distance > 25

    #        points_to_keep2 = temp_dtt >0
    #        points_to_keep2 = list(points_to_keep2)
    #        points_to_keep2.append(1)
    #        points_to_keep2 = np.array(points_to_keep2)

    #        points_to_keep = np.array(points_to_keep1) * points_to_keep2

        ltt = pd.to_numeric(data[i]['RecordingTime [ms]'], errors='coerce')[points_to_keep]
        lxx = pd.to_numeric(data[i]['Point of Regard Right X [px]'][points_to_keep], errors='coerce')
        lyy = pd.to_numeric(data[i]['Point of Regard Right Y [px]'][points_to_keep], errors='coerce')
        lxx = 360/math.pi * np.arctan( (lxx-center_X) / (60*37.795) )
        lyy = 360/math.pi * np.arctan( (lyy-center_Y) / (60*37.795) )
        ltt = 0.001*ltt
        lsacc = np.array(data[i]['Category Right'][points_to_keep])=='Saccade'
        try:
            temp_pupil = np.array(data[i]['Pupil Diameter Right [mm]'])
            temp_pupil = np.array(temp_pupil[temp_pupil!='-']).astype(float)
            pupil[return_index(np.unique(participant_list),participant_list[i])].append(temp_pupil)

            #if len(temp_pupil==1):
            #    pupil[return_index(np.unique(participant_list),participant_list[i])].append(temp_pupil[0])
            #else:
            #    pupil[return_index(np.unique(participant_list),participant_list[i])].append(temp_pupil) 
            ltt_pupil = pd.to_numeric(data[i]['RecordingTime [ms]'], errors='coerce')[temp_pupil!='-']
            TT_pupil[return_index(np.unique(participant_list),participant_list[i])].append(ltt_pupil)
            d_T_pupil[return_index(np.unique(participant_list),participant_list[i])].append(np.diff(ltt_pupil))
        except KeyError:
            0
        
        TT[return_index(np.unique(participant_list),participant_list[i])].append(ltt) 
        XX[return_index(np.unique(participant_list),participant_list[i])].append(lxx) 
        YY[return_index(np.unique(participant_list),participant_list[i])].append(lyy) 
        


        d_X[return_index(np.unique(participant_list),participant_list[i])].append(np.diff(lxx))
        d_Y[return_index(np.unique(participant_list),participant_list[i])].append(np.diff(lyy))
        d_T[return_index(np.unique(participant_list),participant_list[i])].append(np.diff(ltt))
        d_S[return_index(np.unique(participant_list),participant_list[i])].append(np.sqrt(np.diff(lxx)**2 + np.diff(lyy)**2))
        loc_vx = np.diff(lxx) / np.diff(ltt)
        vX[return_index(np.unique(participant_list),participant_list[i])].append( loc_vx[~np.isnan(loc_vx)] )
        loc_vs = np.sqrt(np.diff(lxx)**2 + np.diff(lyy)**2)/ np.diff(ltt)
        vS[return_index(np.unique(participant_list),participant_list[i])].append(loc_vs[~np.isnan(loc_vs)]  )
        saccade[return_index(np.unique(participant_list),participant_list[i])].append(lsacc[:-1][~np.isnan(loc_vs)] )


        dX = d_X
        dY = d_Y
        dS = d_S
        dT = d_T
        
for i in range(len(dX)):
    TT[i] = np.hstack( TT[i])
    XX[i] = np.hstack( XX[i])
    YY[i] = np.hstack( YY[i])
    saccade[i] = np.hstack( saccade[i])


    d_X[i] = np.hstack(d_X[i])
    d_Y[i] = np.hstack(d_Y[i])
    d_T[i] = np.hstack(d_T[i])
    #d_T_pupil[i] = np.hstack(d_T_pupil[i])
    d_S[i] = np.hstack(d_S[i])

    vX[i] = np.hstack(vX[i])

    vS[i] = np.hstack(vS[i])

    dX[i] = np.hstack(dX[i])
    dY[i] = np.hstack(dY[i])
    dS[i] = np.hstack(dS[i])
    dT[i] = np.hstack(dT[i])


for i in range(len(pupil)):
    if len(pupil[i])>0:
        temp_array =[]
        for j in range(len(pupil[i])):
            temp_array.append(np.hstack(pupil[i][j]))
        pupil[i] = np.hstack(temp_array)
        
        

In [ ]:
d_T[0]

In [ ]:
1/0.19

In [ ]:
est_sacc_mu = []
est_sacc_sigma = []
est_fix_mu = []
est_fix_sigma = []
est_lambda_B_list = [] #transition saccade to fix 
est_lambda_D_list = [] # vice versa
guess_logdS_fix_list = []
guess_logdS_sacc_list = []
T_matrix = []
for i in range(len(dS)):
    print(i)
    d__ss = vS[i]
    binary_vector = np.array(saccade[i]).astype(int)
    guess_dS_fix = vS[i][binary_vector==0]
    guess_dS_sacc = vS[i][binary_vector==1]
    guess_logdS_fix = np.log10(guess_dS_fix[guess_dS_fix>0])
    guess_logdS_fix_list.append(guess_logdS_fix)
    guess_logdS_sacc = np.log10(guess_dS_sacc[guess_dS_sacc>0])
    guess_logdS_sacc_list.append(guess_logdS_sacc)
    est_fix_mu.append(np.mean(guess_logdS_fix))
    est_fix_sigma.append(np.std(guess_logdS_fix))
    est_sacc_mu.append(np.mean(guess_logdS_sacc))
    est_sacc_sigma.append(np.std(guess_logdS_sacc))
    Nfix = len(binary_vector) - np.sum(binary_vector)
    Nsacc = np.sum(binary_vector)
    Ntransi = int(np.sum(np.abs(binary_vector[1:] - binary_vector[:-1]))/2)
    T_matrix.append(np.matrix([[1 - Ntransi/Nfix ,Ntransi/Nfix ],[Ntransi/Nsacc ,1 - Ntransi/Nsacc ]]))
    est_lambda_B_list.append(-np.log(Ntransi / Nsacc))
    est_lambda_D_list.append(-np.log(Ntransi / Nfix))
    plt.hist(np.log(guess_dS_fix),np.arange(-1,9,0.1));
    plt.hist(np.log(guess_dS_sacc),np.arange(-1,9,0.1));
    plt.show()

In [ ]:
lalpha=0.5
lcolorASD = 'tab:blue'
lcolorTD = 'dimgrey'
plt.plot(est_fix_mu)
plt.show()
plt.hist(est_fix_mu[separation_threshold2:],np.arange(0.4,1.9,0.1),alpha=lalpha,color=lcolorTD,density=True,label='TD')
plt.hist(est_fix_mu[:separation_threshold2],np.arange(0.4,1.9,0.1),alpha=lalpha,color=lcolorASD,density=True,label='ASD')
plt.xlabel(r'$\mu_F$',fontsize = 12)
plt.ylabel('pdf',fontsize = 12)
plt.show()


#lalpha=1
#lcolorASD = 'blue'
#lcolorTD = 'red'

#lalpha=1
#lcolorASD = 'steelblue'
#lcolorTD = 'coral'


gbins = np.arange(0.3,2,0.02)
kdeTD = stats.gaussian_kde(est_fix_mu[separation_threshold2:])
plt.plot(gbins,kdeTD(gbins),alpha=lalpha,color=lcolorTD,label='TD')
kdeASD = stats.gaussian_kde(est_fix_mu[:separation_threshold2])
plt.plot(gbins,kdeASD(gbins),alpha=lalpha,color=lcolorASD,label='ASD')
plt.xlabel(r'$\mu_F$',fontsize = 14)
plt.ylabel('pdf',fontsize = 14)
plt.legend()
plt.savefig('kde_mu_F.png',dpi=300)
plt.show()




a,b = np.histogram(est_fix_mu[:separation_threshold2],np.arange(0.5,1.9,0.1))
c,d = np.histogram(est_fix_mu[separation_threshold2:],np.arange(0.5,1.9,0.1))
plt.plot((b[1:] + b[:-1])/2,a)
plt.plot((d[1:] + d[:-1])/2,c)
plt.show()
print(np.mean(est_fix_mu[separation_threshold2:]))
print(np.mean(est_fix_mu[:separation_threshold2]))
print(scipy.stats.mannwhitneyu(est_fix_mu[:separation_threshold2], est_fix_mu[separation_threshold2:])[1])
print(scipy.stats.ttest_ind(est_fix_mu[:separation_threshold2], est_fix_mu[separation_threshold2:])[1])





print("---------------------")
plt.plot(est_fix_sigma)
plt.show()
plt.hist(est_fix_sigma[:separation_threshold2],np.arange(0.35,0.55,0.005),alpha=0.6)
plt.hist(est_fix_sigma[separation_threshold2:],np.arange(0.35,0.55,0.005),alpha=0.6)
plt.show()

gbins = np.arange(0.35,0.55,0.005)
kdeTD = stats.gaussian_kde(est_fix_sigma[separation_threshold2:])
plt.plot(gbins,kdeTD(gbins),alpha=lalpha,color=lcolorTD,label='TD')
kdeASD = stats.gaussian_kde(est_fix_sigma[:separation_threshold2])
plt.plot(gbins,kdeASD(gbins),alpha=lalpha,color=lcolorASD,label='ASD')
plt.xlabel(r'$\sigma_F$',fontsize = 14)
plt.ylabel('pdf',fontsize = 14)
plt.legend()
plt.savefig('kde_sigma_F.png',dpi=300)
plt.show()
print(np.mean(est_fix_sigma[separation_threshold2:]))
print(np.mean(est_fix_sigma[:separation_threshold2]))
print(scipy.stats.mannwhitneyu(est_fix_sigma[:separation_threshold2], est_fix_sigma[separation_threshold2:])[1])
print(scipy.stats.ttest_ind(est_fix_sigma[:separation_threshold2], est_fix_sigma[separation_threshold2:])[1])

print("---------------------")
print("Saccades")
plt.plot(est_sacc_mu)
plt.show()
plt.hist(est_sacc_mu[:separation_threshold2],np.arange(0.6,1.9,0.04),alpha=0.6)
plt.hist(est_sacc_mu[separation_threshold2:],np.arange(0.6,1.9,0.04),alpha=0.6)
plt.show()
print(np.mean(est_sacc_mu[separation_threshold2:]))
print(np.mean(est_sacc_mu[:separation_threshold2]))
print(scipy.stats.mannwhitneyu(est_sacc_mu[:separation_threshold2], est_sacc_mu[separation_threshold2:])[1])
print(scipy.stats.ttest_ind(est_sacc_mu[:separation_threshold2], est_sacc_mu[separation_threshold2:])[1])

gbins = np.arange(0.9*min(est_sacc_mu),1.1*max(est_sacc_mu),(max(est_sacc_mu)-min(est_sacc_mu))/50)
kdeTD = stats.gaussian_kde(est_sacc_mu[separation_threshold2:])
plt.plot(gbins,kdeTD(gbins),alpha=lalpha,color=lcolorTD,label='TD')
kdeASD = stats.gaussian_kde(est_sacc_mu[:separation_threshold2])
plt.plot(gbins,kdeASD(gbins),alpha=lalpha,color=lcolorASD,label='ASD')
plt.xlabel(r'$\mu_S$',fontsize = 14)
plt.ylabel('pdf',fontsize = 14)
plt.legend()
plt.savefig('kde_mu_S.png',dpi=300)
plt.show()


print("---------------------")
plt.plot(est_sacc_sigma)
plt.show()
plt.hist(est_sacc_sigma[:separation_threshold2],np.arange(1.2,2,0.02),alpha=0.6)
plt.hist(est_sacc_sigma[separation_threshold2:],np.arange(1.2,2,0.02),alpha=0.6)
plt.show()
print(np.mean(est_sacc_sigma[separation_threshold2:]))
print(np.mean(est_sacc_sigma[:separation_threshold2]))
print(scipy.stats.mannwhitneyu(est_sacc_sigma[:separation_threshold2], est_sacc_sigma[separation_threshold2:])[1])
print(scipy.stats.ttest_ind(est_sacc_sigma[:separation_threshold2], est_sacc_sigma[separation_threshold2:])[1])

gbins = np.arange(0.9*min(est_sacc_sigma),1.1*max(est_sacc_sigma),(max(est_sacc_sigma)-min(est_sacc_sigma))/50)
kdeTD = stats.gaussian_kde(est_sacc_sigma[separation_threshold2:])
plt.plot(gbins,kdeTD(gbins),alpha=lalpha,color=lcolorTD,label='TD')
kdeASD = stats.gaussian_kde(est_sacc_sigma[:separation_threshold2])
plt.plot(gbins,kdeASD(gbins),alpha=lalpha,color=lcolorASD,label='ASD')
plt.xlabel(r'$\sigma_S$',fontsize = 14)
plt.ylabel('pdf',fontsize = 14)
plt.legend()
plt.savefig('kde_sigma_S.png',dpi=300)
plt.show()

print("---------------------")
print("probabilities")
est_P_fix_sacc = []
for i in range(len(dS)):
    est_P_fix_sacc.append(np.swapaxes(T_matrix,0,1)[0][i][1] )
plt.plot(est_P_fix_sacc)
plt.show()
plt.hist(est_P_fix_sacc[separation_threshold2:],np.arange(0.02,0.12,0.005),alpha=0.6)
plt.hist(est_P_fix_sacc[:separation_threshold2],np.arange(0.02,0.12,0.005),alpha=0.6)
plt.show()
print(np.mean(est_P_fix_sacc[separation_threshold2:]))
print(np.mean(est_P_fix_sacc[:separation_threshold2]))
print(scipy.stats.mannwhitneyu(est_P_fix_sacc[:separation_threshold2], est_P_fix_sacc[separation_threshold2:])[1])
print(scipy.stats.ttest_ind(est_P_fix_sacc[:separation_threshold2], est_P_fix_sacc[separation_threshold2:])[1])

gbins = np.arange(0*min(est_P_fix_sacc),1.1*max(est_P_fix_sacc),(max(est_P_fix_sacc)-min(est_P_fix_sacc))/50)
kdeTD = stats.gaussian_kde(est_P_fix_sacc[separation_threshold2:])
plt.plot(gbins,kdeTD(gbins),alpha=lalpha,color=lcolorTD,label='TD')
kdeASD = stats.gaussian_kde(est_P_fix_sacc[:separation_threshold2])
plt.plot(gbins,kdeASD(gbins),alpha=lalpha,color=lcolorASD,label='ASD')
plt.xlabel(r'$P_{FS}$',fontsize = 14)
plt.ylabel('pdf',fontsize = 14)
plt.legend()
plt.savefig('kde_P_FS.png',dpi=300)
plt.show()


print("---------------------")
est_P_sacc_fix = []
for i in range(len(dS)):
    est_P_sacc_fix.append(np.swapaxes(T_matrix,0,1)[1][i][0] )
plt.plot(est_P_sacc_fix)
plt.show()
plt.hist(est_P_sacc_fix[:separation_threshold2],np.arange(0.1,0.6,0.02),alpha=0.6)
plt.hist(est_P_sacc_fix[separation_threshold2:],np.arange(0.1,0.6,0.02),alpha=0.6)
plt.show()
print(np.mean(est_P_sacc_fix[separation_threshold2:]))
print(np.mean(est_P_sacc_fix[:separation_threshold2]))
print(scipy.stats.mannwhitneyu(est_P_sacc_fix[:separation_threshold2], est_P_sacc_fix[separation_threshold2:])[1])
print(scipy.stats.ttest_ind(est_P_sacc_fix[:separation_threshold2], est_P_sacc_fix[separation_threshold2:])[1])

gbins = np.arange(0.9*min(est_P_sacc_fix),1.1*max(est_P_sacc_fix),(max(est_P_sacc_fix)-min(est_P_sacc_fix))/50)
kdeTD = stats.gaussian_kde(est_P_sacc_fix[separation_threshold2:])
plt.plot(gbins,kdeTD(gbins),alpha=lalpha,color=lcolorTD,label='TD')
kdeASD = stats.gaussian_kde(est_P_sacc_fix[:separation_threshold2])
plt.plot(gbins,kdeASD(gbins),alpha=lalpha,color=lcolorASD,label='ASD')
plt.xlabel(r'$P_{SF}$',fontsize = 14)
plt.ylabel('pdf',fontsize = 14)
plt.legend()
plt.savefig('kde_P_SF.png',dpi=300)
plt.show()



In [ ]:
times = np.arange(5,335,5)
times_in_seconds = np.arange(5,335,5)/67
temp_explore_part = []
mean_explore_part = []
for i in range(len(XX)):
    temp_explore_part.append([])
    for t in times:
        occu_matrix=np.histogram2d(XX[i][:t], YY[i][:t], bins=[np.arange(min(XX[i][:t]-10),max(XX[i][:t]+10),0.4),np.arange(min(YY[i][:t]-10),max(YY[i][:t]+10),0.4)],density= False)[0]
        explore_matrix=np.where(occu_matrix != 0, 1, 0)
        temp_explore_part[-1].append(np.sum(explore_matrix))
    mean_explore_part.append(np.mean(np.array(temp_explore_part)/times_in_seconds ))


In [ ]:
(-1)**np.random.randint(0,2,10)

In [ ]:
(-1)**0

In [ ]:
gbins = np.arange(0.9*min(mean_explore_part),1.1*max(mean_explore_part),(max(mean_explore_part)-min(mean_explore_part))/100)
kdeTD = stats.gaussian_kde(mean_explore_part[separation_threshold2:])
#kdeTD.set_bandwidth(0.35)
plt.plot(gbins,kdeTD(gbins),alpha=lalpha,color=lcolorTD,label='TD')
kdeASD = stats.gaussian_kde(mean_explore_part[:separation_threshold2])
#kdeASD.set_bandwidth(0.35)
plt.plot(gbins,kdeASD(gbins),alpha=lalpha,color=lcolorASD,label='ASD')
plt.xlabel(r'$AVA_{\tau}$',fontsize = 14)
plt.ylabel('pdf',fontsize = 14)
plt.legend()
plt.savefig('kde_AVA.png',dpi=300)
plt.show()


In [ ]:
print(np.mean(mean_explore_part[separation_threshold2:]))
print(np.mean(mean_explore_part[:separation_threshold2]))
print(scipy.stats.mannwhitneyu(mean_explore_part[:separation_threshold2], mean_explore_part[separation_threshold2:])[1])
print(scipy.stats.ttest_ind(mean_explore_part[:separation_threshold2], mean_explore_part[separation_threshold2:])[1])

In [ ]:
plt.plot(mean_explore_part)
plt.hlines(max(mean_explore_part[separation_threshold2:]),55,0,color = 'k')
plt.hlines(min(mean_explore_part[:separation_threshold2]),55,0,color = 'k')
#plt.vlines(27,0.5,0.6,color = 'k')

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
import sklearn.metrics 
y_pred = results
y_test = lr_Y



cm=sklearn.metrics.confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation='nearest', cmap="coolwarm")
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=[0,1], yticks=[0,1], xlabel='Predicted label', ylabel='True label')
ax.set_xticklabels(['TD', 'ASD'])
ax.set_yticklabels(['TD', 'ASD'])
for i in range(2):
    for j in range(2):
        ax.text(j, i, format(cm[i, j], 'd'), ha='center', va='center', color='white' if cm[i, j] > cm.max() / 2 else 'black')
plt.savefig('confusion_matrix.png',dpi=400)

accuracy= (cm[1,1]+cm[0,0])/(cm[1,0]+cm[1,1] + cm[0,0]+cm[0,1])
print('Accuracy : ', accuracy)

specificity= cm[0,0]/(cm[0,0]+cm[0,1])
print('Specificity : ', specificity)

sensitivity= cm[1,1]/(cm[1,0]+cm[1,1])
print('Sensitivity : ', sensitivity)

precision= cm[1,1]/(cm[0,1]+cm[1,1])
print('Precision : ', accuracy)





